# 3.马尔可夫决策过程（Markov decision process，MDP）
与多臂老虎机问题不同， **马尔可夫决策过程** 包含状态信息以及状态之间的转移机制

1. 随机过程
2. 马尔可夫过程
3. 马尔可夫奖励过程
4. 马尔可夫决策过程
5. 补充其它概念

导入相关库

In [11]:
import numpy as np

## 3.1 随机过程（stochastic process）
为了明确 **MDP** 的具体定义，需要先了解 **马尔可夫过程和马尔可夫奖励过程** 。而这两个过程都是自发改变的 **随机过程**，所以首先需要知道什么是 **随机过程**：
**随机过程** 是概率论的“动力学”部分。就个人理解是对自然环境中一系列变化的量化描述，这些环境中的变化，即从一个状态到另一个状态的发生是随机的，
每一个状态可以划分在一个固定时刻 *t*，并且将当前时刻的状态信息储存在 **$S_{t}$** 中
同时规定当前状态 **$S_{t}$** 基于历史信息 **$(S_1,\ldots,S_t)$** 变化到另一个状态 **$S_{t+1}$** 的概率为 **$P(S_{t+1}|S_1,\ldots,S_t)$**

## 3.2 马尔可夫过程（Markov process）
当且仅当某时刻的状态只取决于上一时刻的状态时，一个随机过程被称为具有 **马尔可夫性质（Markov property）**
而 **马尔可夫过程** 便是指具有 **马尔可夫性质** 的随机过程，同时也被称为 **马尔可夫链（Markov chain）**
需要注意的是，虽然说 *t+1* 只取决于当前时刻 *t* 的状态，但实际上时刻 *t* 的状态取决于 *t-1* ,通过如此链式的信息传导马尔可夫性可以大大简化运算，因为只要当前状态可知，所有的历史信息都不再需要了，利用当前状态信息就可以决定未来:
$$P(S_{t+1}|S_t)=P(S_{t+1}|S_1,\ldots,S_t)$$

### 一些概念：
可以使用一个元组 $\langle\mathcal{S},\mathcal{P}\rangle$ 来描述一个马尔可夫过程：
$$\mathcal{S}=\{s_1,s_2,\ldots,s_n\}$$
表示的为 **有限个** 状态的集合
$$\mathcal{P}=
\begin{bmatrix}
P(s_1|s_1) & \cdots & P(s_n|s_1) \\
\vdots & \ddots & \vdots \\
P(s_1|s_n) & \cdots & P(s_n|s_n)
\end{bmatrix}$$
则是 **状态转移矩阵（state transition matrix）**
当从某个状态出发，根据 **状态转移矩阵** 可以生成一个 **状态序列（episode）**，这个步骤也被叫做 **采样（sampling）**

## 3.3 马尔可夫奖励过程（Markov reward process,MRP）
**马尔可夫奖励过程** = **马尔可夫过程** + **$\mathrm{R}$（奖励函数）** + **$\gamma$（折扣因子）**
从第时刻$\mathrm{t}$状态$S_{t}$开始，直到终止状态时，所有奖励的衰减之和称为**回报$G_{t}$（Return）：** $$G_t=R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\cdots=\sum_{k=0}^\infty\gamma^kR_{t+k}$$
一个状态的期望回报（即从这个状态出发的**未来累积奖励**的期望）被称为这个状态的**价值**（value）。所有状态的价值就组成了**价值函数**（value function）：
$$\begin{aligned}
V(s) & =\mathbb{E}[G_t|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\ldots|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma(R_{t+1}+\gamma R_{t+2}+\ldots)|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma G_{t+1}|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma V(S_{t+1})|S_t=s] \\
 & =r(s)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s)V(s^{\prime})
\end{aligned}$$
（$s^{\prime}$为下一状态）
上式就是马尔可夫奖励过程中非常有名的**贝尔曼方程（Bellman equation）**
可以看出运算是**分阶段链式传导**的

写成矩阵的形式：
$$\mathcal{V}=\mathcal{R}+\gamma\mathcal{PV} \\
\begin{bmatrix}
V(s_1) \\
V(s_2) \\
\cdots \\
V(s_n)
\end{bmatrix}=
\begin{bmatrix}
r(s_1) \\
r(s_2) \\
\cdots \\
r(s_n)
\end{bmatrix}+\gamma
\begin{bmatrix}
P(s_1|s_1) & P(s_2|s_1) & \ldots & P(s_n|s_1) \\
P(s_1|s_2) & P(s_2|s_2) & \ldots & P(s_n|s_2) \\
\cdots \\
P(s_1|s_n) & P(s_2|s_n) & \ldots & P(s_n|s_n)
\end{bmatrix}
\begin{bmatrix}
V(s_1) \\
V(s_2) \\
\ldots \\
V(s_n)
\end{bmatrix}$$
其中$\mathcal{P}$就为转移概率矩阵

根据矩阵运算可求解析解：
$$\begin{aligned}
\mathcal{V} & =\mathcal{R}+\gamma\mathcal{P}\mathcal{V} \\
(I-\gamma\mathcal{P})\mathcal{V} & =\mathcal{R} \\
\mathcal{V} & =(I-\gamma\mathcal{P})^{-1}\mathcal{R}
\end{aligned}$$

代码实现：

In [12]:
def compute(P, rewards, gamma, states_num):
    """ 利用贝尔曼方程的矩阵形式计算解析解,states_num是MRP的状态数 """
    rewards = np.array(rewards).reshape((-1, 1))  #将rewards写成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P), rewards)
    
    return value

可见以上求解方式的时间复杂度为$O(n^{3})$，n为状态的个数，因此这种方法只适用很小的马尔可夫奖励过程
求解较大规模的马尔可夫奖励过程中的价值函数时，可以使用:
- 1.蒙特卡洛方法（Monte-Carlo method）(4.)
- 2.动态规划（dynamic programming）(5.)
- 3.时序差分（temporal difference）(6.) 

## 3.4 马尔可夫决策过程（Markov decision process，MDP）
之前提到的**马尔可夫过程**和**马尔可夫奖励过程**都是自发改变的随机过程，在此基础上施加**智能体**（agent）动作这个外部因素改变随机过程，就是称之为**马尔可夫决策过程**
用元组表示为:
$$\langle\mathcal{S},\mathcal{A},P,r,\gamma\rangle$$
相较于MRP,多出了$\mathcal{A}$这个所有动作的集合
其中:
- $r(s,a)$是奖励函数，此时奖励可以同时取决于状态s和动作a，在奖励函数只取决于状态s时，则退化为r(s)
- $P(s^{\prime}|s,a)$是状态转移函数，表示在状态s执行动作a之后到达状态s'的概率

由于引入动作这个因素变为三维矩阵，同时许多任务当中的状态不是有限个的，可能是连续的，所以不再使用状态转移矩阵，而是使用状态转移函数


**策略**$  \pi(a|s)=P(A_{t}=a|S_{t}=s)  $是一个函数，表示在输入状态 s 情况下采取动作 a 的概率
一般将**策略**分为以下两种:
1. 确定性策略（deterministic policy）:每个状态时只输出一个确定性的动作，即只有该动作的概率为 1，其他动作的概率为 0；
2. 随机性策略（stochastic policy）:每个状态时输出的是关于动作的概率分布，根据分布进行采样得到一个动作。

智能体采用**不同的策略**在同一状态下所采取的动作会有**不同的概率分布**,所以此时的**价值函数**与**策略**有关
同时因为引入了**动作**，此时的价值函数包括**状态价值函数**（state-value function）与**动作价值函数**（action-value function）两种:
**状态价值函数**:
（一个状态包含所有可能的执行动作，仅确定状态s）
$$\begin{aligned}
V^{\pi}(s) 
 & =\mathbb{E}_\pi[G_t|S_t=s]            \quad (1) 
\end{aligned}$$
**动作价值函数**:
（确定了出发的状态s和动作a）
$$\begin{aligned}
Q^{\pi}(s,a) 
 & =\mathbb{E}_\pi[G_t|S_t=s,A_t=a]                      \quad (2)\\
 & =\mathbb{E}_\pi[R_t+\gamma Q^\pi(S_{t+1},A_{t+1})|S_t=s,A_t=a]  \\
 & =r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)\sum_{a^{\prime}\in A}\pi(a^{\prime}|s^{\prime})Q^{\pi}(s^{\prime},a^{\prime})\quad (3)
\end{aligned}$$


由上述公式（1）（2）易知:
$$V^\pi(s)=\sum_{a\in A}\pi(a|s)Q^\pi(s,a)\quad (4)$$
结合公式（3）（4）:
$$Q^\pi(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^\pi(s^{\prime})\quad (5)$$
结合公式（4）（5）:
$$\begin{aligned}
V^{\pi}(s) 
 & =\sum_{a\in A}\pi(a|s)\left(r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^{\pi}(s^{\prime})\right) \quad (6) 
\end{aligned}$$

**对比MRP的（状态）价值函数:**
$$\begin{aligned}
V(s) & =\mathbb{E}[G_t|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\ldots|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma(R_{t+1}+\gamma R_{t+2}+\ldots)|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma G_{t+1}|S_t=s] \\
 & =\mathbb{E}[R_t+\gamma V(S_{t+1})|S_t=s] \\
 & =r(s)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s)V(s^{\prime})
\end{aligned}$$
之前探讨了 MRP 的解析解方法求状态价值函数，可以将MDP转化为一个 MRP：策略的动作选择进行**边缘化（marginalization)**，即替换：
$$r(s)=\sum_{a\in\mathcal{A}}\pi(a|s)r(s,a)$$
$$P^{\prime}(s^{\prime}|s)=\sum_{a\in\mathcal{A}}\pi(a|s)P(s^{\prime}|s,a)$$
**也就是上面的公式（6）**

公式（6）（3）就是两个价值函数的**贝尔曼期望方程**（Bellman Expectation Equation），加上“期望”二字是为了与接下来的**贝尔曼最优方程**进行区分

以下用代码定义一个简单的马尔可夫决策过程:
![马尔可夫决策过程的一个简单例子](images/马尔可夫决策过程的一个简单例子.png)

In [13]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

实现解析解方法（使用随机策略Pi_1）：

In [14]:
# 直接给出以上MDP转化为MRP的状态转移矩阵P_from_mdp_to_mrp和奖励函数R_from_mdp_to_mrp
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

In [15]:
V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


正如之前讨论的一样：MRP 解析解的方法在状态动作集合比较大的时候**不是很适用**，之后的章节将学习其它方法.

## 3.5 补充其它概念

### 最优策略（optimal policy）：
有限状态和动作集合的 MDP 中，至少存在一个策略比其他**所有**策略都好或者至少存在一个策略不差于其他**所有**策略，这个策略就是**最优策略**
最优策略可能有很多个，都将其表示为$\pi^{*}(s)$
以此引入了**最优状态价值函数**$V^*(s)=\max_\pi V^\pi(s),\quad\forall s\in\mathcal{S}$和**最优动作价值函数**$Q^*(s,a)=\max_\pi Q^\pi(s,a),\quad\forall s\in\mathcal{S},a\in\mathcal{A}$

此时，因为**每阶段**选择的是**最优动作**，所以再得两函数的**关系公式**（7）:
$$V^*(s)=\max_{a\in\mathcal{A}}Q^*(s,a)\quad (7) $$
之前推导出的两函数的**关系公式**（5）：
$$Q^\pi(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in S}P(s^{\prime}|s,a)V^\pi(s^{\prime})\quad (5)$$

### 贝尔曼最优方程（Bellman optimality equation）：
结合两函数的**关系公式**（5）（7），可得**贝尔曼最优方程**：
$$V^*(s)=\max_{a\in\mathcal{A}}\{r(s,a)+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)V^*(s^{\prime})\}\quad (8)$$
$$Q^*(s,a)=r(s,a)+\gamma\sum_{s^{\prime}\in\mathcal{S}}P(s^{\prime}|s,a)\max_{a^{\prime}\in\mathcal{A}}Q^*(s^{\prime},a^{\prime})\quad (9)$$